In [1]:
! pip install transformers datasets -qqq
!pip install transformers datasets evaluate rouge_score -qqq
!pip install --upgrade -qqq wandb

In [2]:
from kaggle_secrets import UserSecretsClient

import wandb

user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("text-summarizer")
secret_value_1 = user_secrets.get_secret("wandb")

In [3]:
import huggingface_hub

wandb.login(key = secret_value_1)

huggingface_hub.login(token=secret_value_0)

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [4]:
from datasets import load_dataset

billsum = load_dataset("billsum", split="ca_test")

Generating train split:   0%|          | 0/18949 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3269 [00:00<?, ? examples/s]

Generating ca_test split:   0%|          | 0/1237 [00:00<?, ? examples/s]

In [5]:
billsum = billsum.train_test_split(test_size=0.2)

In [6]:
from transformers import AutoTokenizer

checkpoint = "Falconsai/text_summarization"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [7]:
prefix = "summarize: "


def preprocess_function(examples):
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [8]:
tokenized_billsum = billsum.map(preprocess_function, batched=True)

Map:   0%|          | 0/989 [00:00<?, ? examples/s]

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

In [9]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

2024-04-21 19:48:04.584774: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-21 19:48:04.584876: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-21 19:48:04.702824: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [10]:
import evaluate

rouge = evaluate.load("rouge")

In [11]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)

    return {k: round(v, 4) for k, v in result.items()}

In [12]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [13]:
training_args = Seq2SeqTrainingArguments(
    output_dir="falcon-summ",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=30,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_billsum["train"],
    eval_dataset=tokenized_billsum["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [14]:
trainer.train()

wandb: Currently logged in as: n-suneel89 (n-suneel-duke). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.659720,0.133600,0.045100,0.110900,0.110900,19.000000
2,No log,2.512576,0.151900,0.058400,0.125800,0.125900,19.000000
3,No log,2.435350,0.170200,0.070500,0.142000,0.142200,19.000000
4,No log,2.385477,0.191500,0.092200,0.161600,0.161800,19.000000
5,No log,2.349905,0.193200,0.093900,0.164100,0.164400,19.000000
6,No log,2.325103,0.195100,0.095900,0.166500,0.166700,19.000000
7,No log,2.303489,0.194300,0.096300,0.165500,0.165600,19.000000
8,No log,2.287869,0.194100,0.095300,0.165200,0.165300,19.000000
9,2.636800,2.269467,0.195200,0.095800,0.166500,0.166700,19.000000
10,2.636800,2.258846,0.194000,0.095200,0.165100,0.165300,19.000000


/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1132: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=1860, training_loss=2.3874246904926917, metrics={'train_runtime': 2124.7079, 'train_samples_per_second': 13.964, 'train_steps_per_second': 0.875, 'total_flos': 8031182499348480.0, 'train_loss': 2.3874246904926917, 'epoch': 30.0})

In [15]:
trainer.push_to_hub()

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

events.out.tfevents.1713728898.c9af55bc8b8e.34.0:   0%|          | 0.00/22.3k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/suneeln-duke/falcon-summ/commit/631c3d5975c4e4623b04eab82a8882eaf6fde185', commit_message='End of training', commit_description='', oid='631c3d5975c4e4623b04eab82a8882eaf6fde185', pr_url=None, pr_revision=None, pr_num=None)

In [16]:
from transformers import pipeline

def predict(text, model):
    
    return summarizer(text)

In [17]:
summarizer = pipeline("summarization", model="suneeln-duke/t5-summ")

config.json:   0%|          | 0.00/1.50k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/20.7k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

In [18]:
text = "summarize: The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It'll lower the deficit and ask the ultra-wealthy and corporations to pay their fair share. And no one making under $400,000 per year will pay a penny more in taxes."

In [19]:
summarizer(text)

Your max_length is set to 200, but your input_length is only 103. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=51)


[{'summary_text': "The Inflation Reduction Act lowers prescription drug costs, health care costs, and energy costs. It's the most aggressive action on tackling the climate crisis in American history, which will lift up American workers and create good-paying, union jobs across the country. It will ask the ultra-wealthy and corporations to pay their fair share."}]

In [24]:
text = """
To the north of the Ganga was the great
kingdom Kosala, made fertile by the river
Sarayu. Its capital was Ayodhya, built by
Manu, the famous ruler of the Solar
dynasty. From Valmiki's description of
the capital Kosala, it is clear that ancient
Ayodhya was not inferior to our modern
cities. Even in ancient India city
civilisation had reached a high level.
King Dasaratha ruled the kingdom
from the capital city of Ayodhya. He had
fought on the side of the Devas, and his
fame spread in the three worlds. He was
the equal of Indra and Kubera. The people
of Kosala were happy, contented and
virtuous. The land was protected by a
mighty army, and no enemy could come
anywhere near
"""

In [21]:
%%time

summarizer(text)

Token indices sequence length is longer than the specified maximum sequence length for this model (799 > 512). Running this sequence through the model will result in indexing errors


CPU times: user 6.83 s, sys: 365 ms, total: 7.2 s
Wall time: 3.6 s


[{'summary_text': 'king Dasaratha ruled the kingdom from the capital city of Ayodhya, and his fame spread in the three worlds. The land contained forts with moats around them as well as many defensive intallations, and true to its name, the land was protected by a mighty army.'}]

In [22]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("summarization", model="Falconsai/text_summarization")

In [25]:
%%time

pipe(text)

CPU times: user 3.87 s, sys: 61.7 ms, total: 3.93 s
Wall time: 1.97 s


[{'summary_text': 'Kosala was fertile by the river Sarayu . Its capital was Ayodhya, built by Manu, the famous ruler of the Solar dynasty . In ancient India city civilisation had reached a high level .'}]